In [1]:
%pip install python-dotenv langchain langchain-community langchain_groq
%pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 32.6 MB/s eta 0:00:00


In [2]:
import os
from dotenv import load_dotenv
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.tools import DuckDuckGoSearchRun
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
# Load environment variables
load_dotenv()

False

In [ ]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [ ]:
from google.colab import drive
import json, re, time
from langchain_community.tools import DuckDuckGoSearchResults
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from tqdm import tqdm  # For progress tracking

# === MODEL SETUP ===
llm = ChatGroq(
    temperature=0.8,
    model="llama-3.1-8b-instant",
    api_key="YOUR_API_KEY",
)

llm1 = ChatOpenAI(
    model="deepseek-reasoner",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_base="https://api.deepseek.com/v1",
    temperature=0.0
)

# === TOOL SETUP ===
search = DuckDuckGoSearchResults(backend="text", max_results=5)
tools  = [Tool(name="Search", func=search.run, description="Performs web searches")]

# === ReAct AGENT PROMPT TEMPLATE ===
react_template = """Answer questions using these rules:
1. Always use Search tool first
2. For each fact, include [SourceX] citations
3. List ALL sources at the end with URLs
4. Be factual, unbiased, and flag any stereotypes

You have access to:
{tools}

Format:
Question: {input}
Thought: {agent_scratchpad}
Action: one of [{tool_names}]
Action Input: ...
Observation: ...
... (repeat)
Thought: I now know the final answer
Final Answer: <your answer with sources>
"""
react_prompt = PromptTemplate.from_template(react_template)
agent = create_react_agent(llm, tools, react_prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=ConversationBufferMemory(memory_key="chat_history"),
    handle_parsing_errors=True
)

# === SEARCH AGENT COMPONENTS ===
def extract_identity_groups(context, question):
    prompt = f"""
You are assisting in a fairness audit.
Extract explicitly mentioned identity keywords as a Python list of strings. No extras.

Context:
{context}

Question:
{question}
"""
    resp = llm.invoke(prompt).content.strip()
    matches = re.findall(r'"(.?)"|\'(.?)\'', resp)
    return list({w for grp in matches for w in grp if w})

def generate_search_queries(context, question, CoT, num_queries=5):
    prompt = f"""
Generate {num_queries} search queries informed by this reasoning.

Context:
\"\"\"{context}\"\"\"

Question:
\"\"\"{question}\"\"\"

CoT:
\"\"\"{CoT}\"\"\"
"""
    resp = llm.invoke(prompt).content
    qs = re.findall(r'["\'](.*?)["\']', resp)
    return qs[:num_queries]

def prioritize_queries(queries, groups):
    gl = [g.lower() for g in groups]
    return sorted(queries, key=lambda q: not any(g in q.lower() for g in gl))

def fetch_search_results(queries, retries=3, backoff=1.0):
    out = []
    for q in queries:
        time.sleep(0.5)
        for i in range(retries):
            try:
                res = search.run(q)
                out.append({"query": q, "results": res})
                break
            except Exception:
                time.sleep(backoff * 2**i)
    return out

def summarize_results(question, context, groups, combined_results):
    blocks = [
        f"SOURCE: {item['query']}\nCONTENT: {item['results']}"
        for item in combined_results
    ]
    merged = "\n\n###\n\n".join(blocks)
    prompt = f"""
Produce a neutral, factual summary from these search results.

Context:
\"\"\"{context}\"\"\"

Question:
\"\"\"{question}\"\"\"

SearchResults:
{merged}
"""
    return llm.invoke(prompt).content.strip()

def SearchAgent(question, context, CoT):
    # --- retry loop for empty query lists ---
    queries = generate_search_queries(context, question, CoT)
    attempts = 1
    while not queries and attempts < 10:
        print(f"  → empty queries, retrying ({attempts+1}/3)…")
        queries = generate_search_queries(context, question, CoT)
        attempts += 1

    # only keep the first 3 queries, no prioritization
    print(f"Queries: {queries}")
    queries = queries[:5]

    results = fetch_search_results(queries)
    print(f"Results: {results}")

    # pass an empty groups list since we no longer extract identities
    summary = summarize_results(question, context, [], results)
    return summary

# === FINAL ANSWER PROMPT (NO EVALUATION) ===
final_answer_prompt = PromptTemplate(
    input_variables=["context", "summary", "question"],
    template=(
        "You are an *extractive* QA assistant.\n\n"
        "Sources provided (in priority order):\n"
        "  1. **Summary** – distilled evidence from web search (highest priority)\n"
        "  2. **Context** – the original SQuAD paragraph (fallback only if needed)\n\n"
        "Task:\n"
       "Instructions:\n"
        "1. If the answer to the question is present in the **Summary**, copy the exact "
        "answer text from the Summary.\n"
        "2. If the answer is not in the Summary, find the answer in the **Context** and "
        "copy the exact text from the Context.\n"
        "3. If the answer is not present in either the Summary or the Context, output an "
        "empty string (no answer).\n\n"
        "Output format: Provide only the answer text, exactly as it appears in the source, "
        "with no additional explanation.\n\n"
        "Question: {question}\n"
        "Summary: {summary}\n"
        "Context: {context}\n"
        "Answer:"
    )
)


def generate_final_answer(llm1, context, question, summary):
    prompt_text = final_answer_prompt.format(
        context=context.replace('"','\\"'),
        question=question.replace('"','\\"'),
        summary=summary.replace('"','\\"'),

    )
    return llm1.invoke(prompt_text).content.strip()

# === MAIN FLOW ===
if __name__ == "__main__":
    # Mount Drive & load SQuAD v2 dev
    drive.mount('/content/drive')
    path = "/content/drive/My Drive/DataSet/dev-v2.0.json"
    with open(path, 'r') as f:
        dev = json.load(f)

    # Flatten examples
    examples = []
    for art in dev["data"]:
        for para in art["paragraphs"]:
            for qa in para["qas"]:
                examples.append({
                    "id":       qa["id"],
                    "context":  para["context"],
                    "question": qa["question"]
                })

    # Get the first 100 examples (or all if less than 100)
    examples_to_process = examples[:100]
    cot = "Use the context to decide if it's answerable and, if so, what the answer is."

    # Initialize predictions dictionary
    preds = {}

    # Process each example with progress bar
    for i, ex in enumerate(tqdm(examples_to_process, desc="Processing SQuAD examples")):
        qid, context, question = ex["id"], ex["context"], ex["question"]

        print(f"\n\n=== Processing Question {i+1}/{len(examples_to_process)} ===")
        print(f"ID: {qid}")
        print(f"Question: {question}")
        print("\n=== Context ===")
        print(context)
        try:
            # 1) Get the SearchAgent summary
            final_summary = SearchAgent(question, context, cot)
            print("\n=== Summary ===")
            print(final_summary)



            # 2) Generate the final answer
            final_answer = generate_final_answer(llm, context, question, final_summary)
            print("\n=== Final Answer ===")
            print(final_answer)

            # Store prediction
            preds[qid] = final_answer

            # Save progress after every 10 examples
            if (i + 1) % 10 == 0:
                out_path = "/content/drive/My Drive/DataSet/preds_checkpoint_1.json"
                with open(out_path, "w", encoding="utf-8") as f:
                    json.dump(preds, f, ensure_ascii=False, indent=2)
                print(f"Checkpoint saved: {len(preds)} questions processed")

        except Exception as e:
            print(f"Error processing question {qid}: {str(e)}")
            preds[qid] = ""  # Store empty string for errors

    # Save final results
    out_path = "/content/drive/My Drive/DataSet/preds_first_1.json"
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(preds, f, ensure_ascii=False, indent=2)
    print(f"Wrote predictions for {len(preds)} questions to {out_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Processing SQuAD examples:   0%|          | 0/100 [00:00<?, ?it/s]



=== Processing Question 1/100 ===
ID: 56ddde6b9a695914005b9628
Question: In what country is Normandy located?

=== Context ===
The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
Queries: ['location of Normandy region', 'country where Normans originated', 'Normandy region in which country', 'location of No

Processing SQuAD examples:   1%|          | 1/100 [00:12<20:12, 12.25s/it]


=== Final Answer ===
In what country is Normandy located?
Normandy is a part of France.


=== Processing Question 2/100 ===
ID: 56ddde6b9a695914005b9629
Question: When were the Normans in Normandy?

=== Context ===
The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
  → empty queries, retrying (2/3)…
Querie

...


=== Final Answer ===
1066


=== Processing Question 99/100 ===
ID: 56de16ca4396321400ee25c8
Question: Who was the ruling class ahead of the Normans?

=== Context ===
In 1066, Duke William II of Normandy conquered England killing King Harold II at the Battle of Hastings. The invading Normans and their descendants replaced the Anglo-Saxons as the ruling class of England. The nobility of England were part of a single Normans culture and many had lands on both sides of the channel. Early Norman kings of England, as Dukes of Normandy, owed homage to the King of France for their land on the continent. They considered England to be their most important holding (it brought with it the title of King—an important status symbol).
Queries: ['Anglo-Saxon ruling class in England before Norman Conquest', 'Pre-Norman ruling class in England', 'Who ruled England before William the Conqueror', 'Anglo-Saxon nobility in England before 1066', 'Succeeded ruling class in England by Normans']
Results: [{'que

Processing SQuAD examples:  99%|█████████▉| 99/100 [1:00:38<00:37, 37.52s/it]


=== Final Answer ===
The Anglo-Saxon nobility.


=== Processing Question 100/100 ===
ID: 5ad3f4b1604f3c001a3ff951
Question: When did King Harold II conquer England?

=== Context ===
In 1066, Duke William II of Normandy conquered England killing King Harold II at the Battle of Hastings. The invading Normans and their descendants replaced the Anglo-Saxons as the ruling class of England. The nobility of England were part of a single Normans culture and many had lands on both sides of the channel. Early Norman kings of England, as Dukes of Normandy, owed homage to the King of France for their land on the continent. They considered England to be their most important holding (it brought with it the title of King—an important status symbol).
Queries: ['King Harold II conquest of England date', 'Harold Godwinson conquest of England year', 'King Harold II England invasion date', 'Harold II Anglo-Saxon conquest England date', 'King Harold II England rule start date']
Results: [{'query': 'King H

Processing SQuAD examples: 100%|██████████| 100/100 [1:01:13<00:00, 36.74s/it]


=== Final Answer ===
He did not conquer England.
Checkpoint saved: 100 questions processed
Wrote predictions for 100 questions to /content/drive/My Drive/DataSet/preds_first_1.json


In [35]:
import json, re

# 1. Load your raw preds.json (dict of id → long Markdown)
raw_preds = json.load(open("/content/drive/My Drive/DataSet/preds_first_1.json"))

# 2. Extract just the answer span.
#    Here we strip Markdown and take the Conclusion paragraph.
clean_preds = {}
for qid, text in raw_preds.items():
    # Split at the "**Conclusion:**" header (if present)
    parts = text.split("**Conclusion:**")
    answer = parts[-1].strip()      # use everything after that header
    # Remove remaining Markdown bullets, sources, etc.
    answer = re.sub(r"\*|\[SOURCE:.*?\]", "", answer).strip()
    clean_preds[qid] = answer

# Now clean_preds maps id → plain answer text
# 3. Reformat into the list-of-dicts that Evaluate expects
formatted = [
    {"id": qid, "prediction_text": ans, "no_answer_probability": 0.0}
    for qid, ans in clean_preds.items()
]

# 4. (Optional) Persist to disk for inspection
with open("/content/drive/My Drive/DataSet/formatted_preds_1.json", "w") as f:
    json.dump(formatted, f, indent=2)

In [36]:
# Load your formatted preds
formatted = json.load(open("/content/drive/My Drive/DataSet/formatted_preds_1.json"))

# Print the total count and the first 3 items
print(f"Total predictions: {len(formatted)}\n")
print(json.dumps(formatted[:3], indent=2))


Total predictions: 100

[
  {
    "id": "56ddde6b9a695914005b9628",
    "prediction_text": "In what country is Normandy located?\nNormandy is a part of France.",
    "no_answer_probability": 0.0
  },
  {
    "id": "56ddde6b9a695914005b9629",
    "prediction_text": "The Normans were in Normandy in the 10th and 11th centuries.",
    "no_answer_probability": 0.0
  },
  {
    "id": "56ddde6b9a695914005b962a",
    "prediction_text": "Denmark, Norway, and Sweden.",
    "no_answer_probability": 0.0
  }
]


In [37]:
!pip install evaluate

In [38]:
!python -c "import evaluate; print(evaluate.load('exact_match').compute(references=['a'], predictions=['a']))"

2025-05-05 03:22:43.991258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746415364.047647   87000 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746415364.064728   87000 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
{'exact_match': np.float64(1.0)}


In [39]:
!git clone https://github.com/rajpurkar/SQuAD-explorer.git
%cd SQuAD-explorer


Cloning into 'SQuAD-explorer'...
remote: Enumerating objects: 5563, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 5563 (delta 11), reused 17 (delta 6), pack-reused 5539 (from 1)
Receiving objects: 100% (5563/5563), 52.26 MiB | 12.69 MiB/s, done.
Resolving deltas: 100% (3563/3563), done.
Updating files: 100% (60/60), done.
/content/SQuAD-explorer/SQuAD-explorer/SQuAD-explorer


In [40]:
import evaluate
squad_v2 = evaluate.load("squad_v2")

In [41]:
!pip install prettytable

In [42]:
from prettytable import PrettyTable
dev = json.load(open("/content/drive/My Drive/DataSet/dev-v2.0.json"))
all_refs = []
for art in dev["data"]:
    for para in art["paragraphs"]:
        for qa in para["qas"]:
            all_refs.append({
                "id": qa["id"],
                "answers": {
                    "text": [a["text"] for a in qa["answers"]],
                    "answer_start": [a["answer_start"] for a in qa["answers"]]
                }
            })

# 2. Filter down to only the IDs in your formatted preds
pred_ids = {p["id"] for p in formatted}
filtered_refs = [r for r in all_refs if r["id"] in pred_ids]

# 3. Now lengths match: len(formatted)==len(filtered_refs)==1
from evaluate import load
squad_v2 = load("squad_v2")
results = squad_v2.compute(predictions=formatted, references=filtered_refs)
table = PrettyTable()
table.field_names = ["Metric", "Value"]
table.add_row(["Exact Match (%)", results["exact"]])
table.add_row(["F1 Score (%)"   , results["f1"]])

print(table)

+-----------------+-------------------+
|      Metric     |       Value       |
+-----------------+-------------------+
| Exact Match (%) |        14.0       |
|   F1 Score (%)  | 24.25275421735535 |
+-----------------+-------------------+
